In [1]:
# インポート
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

print('numpy version:', np.__version__)
print('tensorflow version:', tf.__version__)
print('GPU:', tf.test.gpu_device_name())

SyntaxError: invalid syntax (pywrap_tensorflow_internal.py, line 114)

In [0]:
# データ読み込み関数
def loadData(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1,
                      unpack=False, dtype=str)
    speed = []
    xyzwave = []
    for row in data:
        speed.append(row[1].astype('f8'))
        xyzwave.append(row[2: 2 + 3 * 50].astype('f8'))
    speed = np.array(speed)
    x = np.array(xyzwave)
    t = speed[:, np.newaxis]
    return x, t

In [0]:
# 入力・教師データを対応させたままシャッフル
def shuffleData(x, t):
    zipped = list(zip(x, t))
    np.random.shuffle(zipped)
    x_result, t_result = zip(*zipped)
    return np.asarray(x_result), np.asarray(t_result)

In [0]:
# 学習データとテストデータに分割
def splitData(x, t):
    border = int(len(x) * 0.9)
    return (x[:border], t[:border]), (x[border:], t[border:])

In [0]:
# データの読み込み
x1, t1 = loadData('data/ML/0912_1800.csv')
x2, t2 = loadData('data/ML/0912_1815.csv')
x3, t3 = loadData('data/ML/0925.csv')
x4, t4 = loadData('data/ML/1010.csv')
x_all = np.concatenate([x1, x2, x3, x4], axis=0)
t_all = np.concatenate([t1, t2, t3, t4], axis=0)

x, t = loadData('data/ML/1010.csv')

x_shuffle, t_shuffle = shuffleData(x, t)
(x_train, t_train), (x_test,  t_test) = splitData(x_shuffle, t_shuffle)

In [0]:
# モデル生成
model = keras.Sequential()
model.add(layers.Input(shape=(150,)))
model.add(layers.Dense(100, activation='tanh'))
model.add(layers.Dense( 70, activation='tanh'))
model.add(layers.Dense( 40, activation='tanh'))
model.add(layers.Dense( 10, activation='tanh'))
model.add(layers.Dense(  1))

adam = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='mse')

In [0]:
# モデルの訓練
result = model.fit(x_train, t_train, epochs=100)

In [0]:
# 学習データでテスト
y_train = model.predict(x_train)
error_train = np.abs(t_train - y_train)

# テストデータでテスト
y_test = model.predict(x_test)
error_test  = np.abs(t_test - y_test)

In [0]:
# 誤差率表示
errorRate_train = np.average(error_train) / np.average(t_train) * 100
errorRate_test  = np.average(error_test ) / np.average(t_test) * 100
print('error_train_average = {} [%]'.format(errorRate_train))
print('error_test_average  = {} [%]'.format(errorRate_test))

In [0]:
# グラフ表示
import matplotlib.pyplot as plt

# matplotlib設定
plt.rcParams['font.family'] = ['Yu Gothic', 'Corporate Logo Medium']

fig, axes = plt.subplots(3, 1, figsize=(10,10))

axes[0].set_title('LOSS')
axes[0].set_xlabel('epochs')
axes[0].set_ylabel('loss')
axes[0].plot(result.history['loss'], color='blue')
axes[0].grid(color='gray')

axes[1].set_title('train data')
axes[1].set_xlabel('datanum')
axes[1].set_ylabel('speed [m/s]')
axes[1].plot(t_train, label='teacher', color='gray')
axes[1].plot(y_train, label='output', color='blue', alpha=0.7)
axes[1].plot(error_train, label='error', color='red', alpha=0.7)
axes[1].legend()
axes[1].grid(color='gray')

axes[2].set_title('test data')
axes[2].set_xlabel('datanum')
axes[2].set_ylabel('speed [m/s]')
axes[2].plot(t_test, label='teacher', color='gray')
axes[2].plot(y_test, label='output', color='blue', alpha=0.7)
axes[2].plot(error_test, label='error', color='red', alpha=0.7)
axes[2].legend()
axes[2].grid(color='gray')

# 日本語化
# axes[0].set_title('損失関数')
# axes[1].set_title('学習データ')
# axes[2].set_title('テストデータ')

fig.subplots_adjust(hspace=0.5)
fig.align_labels()
fig.show()
fig.savefig('graph/graph.png')